In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

from keras_vggface.vggface import VGGFace

from utils.DataLoader import load_data
from models.SiameseNet import SiameseModel, build_network
from models.Attention import CBAM

import numpy as np
import matplotlib.pyplot as plt

from IPython.display import Image, display
import matplotlib.cm as cm

In [2]:
def load_datasets(dataset, test_fold, batch_size=128):
    if dataset == 'KI':
        train_dataset = load_data(f'data\\KinFaceWITrainFolds{test_fold}.tfrecords', batch_size=batch_size)
        validation_dataset = load_data(f'data\\KinFaceWITestFolds{test_fold}.tfrecords', batch_size=batch_size)
    elif dataset == 'KII':
        train_dataset = load_data(f'data\\KinFaceWIITrainFolds{test_fold}.tfrecords', batch_size=batch_size)
        validation_dataset = load_data(f'data\\KinFaceWIITestFolds{test_fold}.tfrecords', batch_size=batch_size)
        
    
    return train_dataset, validation_dataset

In [3]:
class MaskModel(keras.models.Model):
    def __init__(self, mask_model, alpha=.5):
        super().__init__()
        
        self.mask_model = mask_model
        self.alpha = alpha
    
    def call(self, x):
        mask = self.mask_model(x)
        mask = tf.argmax(mask, -1)
        mask = tf.expand_dims(mask, 3)
        mask = tf.cast(mask, tf.float32)
        
        x = self.alpha * mask + (1 - self.alpha) * x
        
        return x

In [4]:
def build_siamese_network(use_cbam, train_vgg, embedding_size):
    input_layer = layers.Input((64, 64, 3))
    vgg = VGGFace(model='vgg16', include_top=False, input_shape=(64, 64, 3))
    
    if use_cbam:
        cbam = CBAM(3)(input_layer)
        x = vgg(cbam)
    else:
        x = vgg(input_layer)
        
    x = layers.Flatten()(x)
    
    output_layer = layers.Dense(embedding_size)(x)

    network = keras.Model(inputs=input_layer, outputs=output_layer)
    
    if not train_vgg:
        for layer in vgg.layers:
            layer.trainable = False
        
    return network

def add_prior_attention_to_model(base_model):
    mask_generator = keras.models.load_model('model.h5')
    mask_model = MaskModel(mask_generator, .2)
    
    input_layer = layers.Input((64, 64, 3))
    x = mask_model(input_layer)
    x = base_model(x)
    
    model = keras.models.Model(inputs=input_layer, outputs=x)
    
    for layer in mask_generator.layers:
        layer.trainable = False
    
    return model

def build_final_model(use_prior=False, use_cbam=True, train_vgg=False, embedding_size=1024):
    network = build_siamese_network(use_cbam, train_vgg, embedding_size)
    
    if use_prior:
        network = add_prior_attention_to_model(network)
        
    siamese_network = build_network(network)
    model = SiameseModel(siamese_network)
    
    return model, network

In [5]:
def train_model(model, train_dataset, validation_dataset):
    model.compile(optimizer=keras.optimizers.Adam(0.001))

    callbacks = [
        tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20),
        tf.keras.callbacks.CSVLogger('.\\log.csv', separator=",", append=True),
        tf.keras.callbacks.ReduceLROnPlateau(
                monitor='val_accuracy',
                factor=0.1,
                patience=5,
                verbose=1,
                min_lr=1e-7
        )
    ]

    history = model.fit(train_dataset, epochs=1, validation_data=validation_dataset, callbacks=callbacks)
    
    return history

In [6]:
# 93.98 --> Prior, CBAM, batch_size=32

In [7]:
def log_config(use_prior, use_cbam, train_vgg, embedding_size, dataset, fold, batch_size=16):
    train_dataset, validation_dataset = load_datasets(dataset, fold, batch_size)
    model, network = build_final_model(use_prior, use_cbam, train_vgg, embedding_size)
    history = train_model(model, train_dataset, validation_dataset)
    
    del network, model, train_dataset, validation_dataset
    return history

In [8]:
# batch_sizes = [16]
# embedding_sizes = [1024] 
use_prior_list = [True, False]
use_cbam_list = [True, False] 
train_vgg_list = [False, True]
datasets = ['KI', 'KII']
folds = ['0', '1', '2', '3', '4']

logs = []
counter = 0

methods = {
    (True, True, False, 'KI', '0'),
    (True, True, False, 'KI', '1'),
    (True, True, False, 'KI', '2'),
    (True, True, False, 'KI', '3'),
    (True, True, False, 'KI', '4'),
    
    (True, True, False, 'KII', '0'),
    (True, True, False, 'KII', '1'),
    (True, True, False, 'KII', '2'),
    (True, True, False, 'KII', '3'),
    (True, True, False, 'KII', '4'),
    
    (False, True, False, 'KI', '0'),
    (False, True, False, 'KI', '1'),
    (False, True, False, 'KI', '2'),
    (False, True, False, 'KI', '3'),
    (False, True, False, 'KI', '4'),
    
    (False, True, False, 'KII', '0'),
    (False, True, False, 'KII', '1'),
    (False, True, False, 'KII', '2'),
    (False, True, False, 'KII', '3'),
    (False, True, False, 'KII', '4'),
    
    (True, False, False, 'KI', '0'),
    (True, False, False, 'KI', '1'),
    (True, False, False, 'KI', '2'),
    (True, False, False, 'KI', '3'),
    (True, False, False, 'KI', '4'),
    
    (True, False, False, 'KII', '0'),
    (True, False, False, 'KII', '1'),
    (True, False, False, 'KII', '2'),
    (True, False, False, 'KII', '3'),
    (True, False, False, 'KII', '4'),
    
    (False, False, False, 'KI', '0'),
    (False, False, False, 'KI', '1'),
    (False, False, False, 'KI', '2'),
    (False, False, False, 'KI', '3'),
    (False, False, False, 'KI', '4'),
    
    (False, False, False, 'KII', '0'),
    (False, False, False, 'KII', '1'),
    (False, False, False, 'KII', '2'),
    (False, False, False, 'KII', '3'),
    (False, False, False, 'KII', '4'),
    
    (True, True, True, 'KI', '0'),
    (True, True, True, 'KI', '1'),
    (True, True, True, 'KI', '2'),
    (True, True, True, 'KI', '3'),
    (True, True, True, 'KI', '4'),
    
    (True, True, True, 'KII', '0'),
    
    (True, False, True, 'KI', '0'),
    
    
    (True, False, True, 'KII', '0'),
    
    
    (False, False, True, 'KI', '0'),
    
    
    (False, False, True, 'KII', '0'),
    
    
    (False, True, True, 'KI', '0'),
    
    
    (False, True, True, 'KII', '0'),
    
}
for train_vgg in train_vgg_list:
    for use_cbam in use_cbam_list:
        for use_prior in use_prior_list:
            for dataset in datasets:
                for fold in folds:
                    
                    config = {
                        'use_prior': use_prior,
                        'use_cbam': use_cbam,
                        'train_vgg': train_vgg,
                        'dataset': dataset,
                        'test_fold': fold
                    }

                    method = (use_prior, use_cbam, train_vgg, dataset, fold)
                    if method in methods:
                        continue
                    else:
                        methods.add(method)

                    print(config)
#                     use_prior, use_cbam, train_vgg, embedding_size, dataset, fold, batch_size=16
                    history = log_config(use_prior, use_cbam, train_vgg, 1024, dataset, fold, batch_size=16)
                    
                    
                    config['history'] = history
                    logs.append(config)

{'use_prior': True, 'use_cbam': True, 'train_vgg': True, 'dataset': 'KII', 'test_fold': '1'}
44925/44925 [==============================] - 5199s 116ms/step - loss: 0.4997 - accuracy: 0.9966 - ap_mean: 0.0017 - ap_std: 147817120.0000 - an_mean: 38246668.0000 - an_std: 147817120.0000 - val_loss: 0.5000 - val_accuracy: 1.0000 - val_ap_mean: 0.0000e+00 - val_ap_std: 0.0000e+00 - val_an_mean: 0.0000e+00 - val_an_std: 0.0000e+00
{'use_prior': True, 'use_cbam': True, 'train_vgg': True, 'dataset': 'KII', 'test_fold': '2'}
79900/79900 [==============================] - 9002s 113ms/step - loss: 6095.1963 - accuracy: 0.9891 - ap_mean: 33705.8711 - ap_std: 471295.5000 - an_mean: 405156.9062 - an_std: 471295.5000 - val_loss: 0.5000 - val_accuracy: 1.0000 - val_ap_mean: 0.0000e+00 - val_ap_std: 0.0000e+00 - val_an_mean: 0.0000e+00 - val_an_std: 0.0000e+00
{'use_prior': True, 'use_cbam': True, 'train_vgg': True, 'dataset': 'KII', 'test_fold': '3'}
79900/79900 [==============================] - 9031s

ResourceExhaustedError:  OOM when allocating tensor with shape[512,512,3,3] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node gradient_tape/model_39/model_38/model_37/vggface_vgg16/conv5_1/Conv2D_1/Conv2DBackpropFilter (defined at D:\Kinship Verification\tensorflow_codes\models\SiameseNet.py:73) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_4779478]

Errors may have originated from an input operation.
Input Source operations connected to node gradient_tape/model_39/model_38/model_37/vggface_vgg16/conv5_1/Conv2D_1/Conv2DBackpropFilter:
 model_39/model_38/model_37/vggface_vgg16/pool4/MaxPool_1 (defined at D:\Kinship Verification\tensorflow_codes\models\SiameseNet.py:111)

Function call stack:
train_function


In [ ]:
logs

In [ ]:
import pickle

In [ ]:
file_log = []

for item in logs:
    file_log.append({
        'use_prior': item['use_prior'],
        'use_cbam': item['use_cbam'],
        'train_vgg': item['train_vgg'],
        'embedding_size': item['embedding_size'],
        'dataset': item['dataset'],
        'batch_size': item['batch_size'],
        'logs': item['history'].history
    })


In [ ]:
byte_array = pickle.dumps(file_log)

In [ ]:
with open('ablation_study_final.pkl', 'wb') as file:
    file.write(byte_array)

In [ ]:
# config = {'use_prior': True, 'use_cbam': True, 'train_vgg': True, 'embedding_size': 1024, 'dataset': 'KI', 'batch_size': 16}

In [ ]:
# history = log_config(**config)

In [ ]:
# for item in file_log:
#     print(f'prior: {item["use_prior"]}, use cbam:{item["use_cbam"]}, train_vgg: {item["train_vgg"]}, embedding_size:{item["embedding_size"]}, dataset: {item["dataset"]}')
#     print(item['logs']['val_accuracy'])
#     print()

In [ ]:
# logs[0]['history'].history['val_accuracy']

In [ ]:
# fig, axs = plt.subplots(6, 3, figsize=(15, 12))
# fig.tight_layout(pad=2.0)

# for i in range(6):
#     for j in range(3):
#         idx = i * 3 + j
#         item = file_log[idx]
#         accuracy = item['logs']['accuracy']
#         val_accuracy = item['logs']['val_accuracy']
        
#         axs[i, j].plot(accuracy, color='blue', label='accuracy')
#         axs[i, j].plot(val_accuracy, color='red', label='val_accuracy')
#         axs[i, j].legend()
        
#         label = f'{item["use_prior"]}, {item["use_cbam"]}, {item["train_vgg"]}, {item["dataset"]}, {item["embedding_size"]}, {val_accuracy[-1]:.2f}'
        
#         axs[i, j].set_title(label)
# plt.show()

In [ ]:
# mask_generator = keras.models.load_model('model.h5')

In [ ]:
# keras.utils.plot_model(mask_generator)